<a href="https://colab.research.google.com/github/bahadirbasaran/pulsarDetection/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Environment**

In [0]:
import copy
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

**Function&Class Definitions**

In [0]:
def createDataSplits(dataset, list_dataColumns, targetColumn, testSizeRatio):
    
    X = dataset[dataset.columns[list_dataColumns]]
    y = dataset[dataset.columns[targetColumn]]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testSizeRatio, stratify=y, random_state=42)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test

**Data Importation**

In [0]:
pulsarData = pd.read_csv('https://raw.githubusercontent.com/bahadirbasaran/pulsarDetection/master/pulsar_stars.csv?token=ALRHARXNN6L47R4KLJOGKP26LQWMU')

**Data Visualization**

In [0]:
pulsarData.info()
datasetStats = pulsarData.describe()

# Correlation Matrix Heatmap
_, ax = plt.subplots(figsize = (10,10))
hm = sb.heatmap(pulsarData.corr(), annot = True, linewidths=.5, fmt = ".3f", ax=ax)
bottom, top = hm.get_ylim()
hm.set_ylim(bottom + 0.5, top - 0.5)
plt.show()

# Pair Plot
g = sb.pairplot(pulsarData)

# Bar Plot
_, ax1 = plt.subplots(3,3, figsize=(15,10))
k = 0
for i in range(3):
    for j in range(3):
        if k != 10:
            sb.barplot(x='target_class', y=pulsarData.iloc[:,k], data=pulsarData, ax = ax1[i][j])
            k += 1            
plt.show()


**bold text**

In [0]:
X_train, X_test, y_train, y_test = createDataSplits(pulsarData, [0,1,2,3,4,5,6,7], 8, 0.3)
